# Question
Write query to remove duplicate rows based on EmployeeID.
Can you write query using window functions?

# Setting up test environment
- Creating the table
- Loading the data

In [1]:
import psycopg2
import csv

conn = psycopg2.connect(
    host="192.168.1.109",
    database="ria",
    user="postgres",
    password="01570376"
)

cur = conn.cursor()

cur.execute("SELECT EXISTS (SELECT 1 FROM pg_tables WHERE tablename = 'employee')")
table_exists = cur.fetchone()[0]

if not table_exists:

    cur.execute('''
        CREATE TABLE employee (
            EmployeeID INTEGER,
            EmployeeName VARCHAR(255),
            Salary NUMERIC(10,2),
            ManagerID INTEGER
        )
    ''')
else:
    cur.execute('TRUNCATE TABLE employee')


with open('employee.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip the header row
    cur.copy_from(
        file=f,
        table='employee',
        sep=',',
        null='',
        columns=('employeeid', 'employeename', 'salary', 'managerid')
    )

# Commit the changes and close the cursor and connection
conn.commit()
cur.close()
conn.close()

# Showing how data looks currently

In [2]:
import pandas as pd
import psycopg2

# Establish a connection to the database
conn = psycopg2.connect(
    host="192.168.1.109",
    database="ria",
    user="postgres",
    password="01570376"
)

# Create a pandas dataframe from the SQL query results
sql_query = '''
SELECT * FROM public.employee
'''
df = pd.read_sql_query(sql_query, conn)
df

/tmp/ipykernel_15078/1462126047.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


,employeeid,employeename,salary,managerid
0,1001,Sarah,90000.0,1002.0
1,1002,Jim,105000.0,NaN
2,1003,Henry,91000.0,1001.0
3,1004,Jenny,90000.0,1002.0
4,1005,Joe,95000.0,1003.0
5,1001,Sarah,90000.0,1002.0
6,1002,Jim,105000.0,NaN
7,1003,Henry,91000.0,1001.0
8,1004,Jenny,90000.0,1002.0
9,1005,Joe,95000.0,1003.0


# ANSWER:

In [3]:
conn = psycopg2.connect(
    host="192.168.1.109",
    database="ria",
    user="postgres",
    password="01570376"
)
cur = conn.cursor()

sql_query = '''
DELETE FROM employee
WHERE ctid IN (
  SELECT ctid
  FROM (
    SELECT ctid, ROW_NUMBER() OVER (PARTITION BY EmployeeID ORDER BY ctid) AS rn
    FROM employee
  ) x
  WHERE rn > 1
);

'''

cur.execute(sql_query)

conn.commit()
cur.close()
conn.close()


ctid is a system column that creates a unique id for each row. This is important so that when deleting I just delete the duplicates and not the originals which have the same id as the duplicates.

# Showing the results

In [4]:
import pandas as pd
import psycopg2

# Establish a connection to the database
conn = psycopg2.connect(
    host="192.168.1.109",
    database="ria",
    user="postgres",
    password="01570376"
)

# Create a pandas dataframe from the SQL query results
sql_query = '''
SELECT * FROM public.employee
'''
df = pd.read_sql_query(sql_query, conn)
df

/tmp/ipykernel_15078/1462126047.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


,employeeid,employeename,salary,managerid
0,1001,Sarah,90000.0,1002.0
1,1002,Jim,105000.0,NaN
2,1003,Henry,91000.0,1001.0
3,1004,Jenny,90000.0,1002.0
4,1005,Joe,95000.0,1003.0
